In [1]:
from gensim.models import Word2Vec
import pandas as pd
import nltk
from tqdm import tqdm
import numpy as np
import string

In [2]:
Cell_Phones_and_Accessories = pd.read_json('~/Documents/Msc-project/data/reviews_Cell_Phones_and_Accessories_5.json', lines=True)

In [3]:
stopwords=[]
with open('stopwords.txt') as file:
    for line in file.readlines():
        stopwords.append(line.strip())

In [4]:
sentences = []
with tqdm(total = len(Cell_Phones_and_Accessories)) as pbar:
    for review in Cell_Phones_and_Accessories.iloc[:,3]:
        review = nltk.sent_tokenize(review)
        for i in range(len(review)):
            review[i] = review[i].lower()
            for punc in string.punctuation:
                review[i] = review[i].replace(punc, ' ')
        sentences.append(review)
        pbar.update(1)

100%|██████████| 194439/194439 [00:51<00:00, 3745.49it/s]


In [8]:
s = []
for sentence in sentences:
    for ss in sentence:
        s.append(ss)
print len(s)

1001122


In [9]:
with tqdm(total = len(s)) as pbar:
    for i in range(len(s)):
        s[i] = nltk.word_tokenize(s[i])
        pbar.update(1)

100%|██████████| 1001122/1001122 [02:10<00:00, 7661.47it/s]


In [10]:
with tqdm(total = len(s)) as pbar:
    for i in range(len(s)):
        new_sent = []
        for word in s[i]:
            if word not in stopwords and len(word)>2:
                new_sent.append(word)
        s[i] = new_sent
        pbar.update(1)
print s[0:2]

100%|██████████| 1001122/1001122 [07:17<00:00, 2289.03it/s]

[[u'look', u'good', u'stick', u'good'], [u'just', u'don', u'like', u'rounded', u'shape', u'always', u'bumping', u'siri', u'kept', u'popping', u'irritating']]


In [11]:
lens = [len(sen) for sen in s]
print np.mean(lens)

8.79212523549


In [12]:
model = Word2Vec(s, size=100, window=5, min_count=5, workers=4)

In [13]:
model.save('word2vec-model')

In [14]:
keys = model.wv.vocab.keys()

In [15]:
key_dict = {}
for key in keys:
    key_dict[key] = 0
for sentence in s:
    for word in sentence:
        if key_dict.has_key(word):
            key_dict[word] += 1
key_list = sorted(key_dict.items(), key=lambda x:x[1], reverse=True)

In [23]:
import multiprocessing
import sys
import time
def pros(review):
    review = review.lower()
    for punc in string.punctuation:
        review = review.replace(punc, ' ')
    review = nltk.word_tokenize(review)
    feature_vector = np.zeros(1000)
    feature_dict = {}
    for feature in features:
        feature_dict[feature] = 0
    for word in review:
        similar_words = []
        if word not in stopwords and len(word)>2:
            if model.wv.vocab.has_key(word):
                similar_words = model.most_similar(word)
        for similar_word_pair in similar_words:
            similar_word = u''+similar_word_pair[0]
            if feature_dict.has_key(similar_word):
                feature_dict[similar_word] += 1
    for i in range(len(features)):
        feature_vector[i] = feature_dict[features[i]]
    feature_matrix.append(feature_vector)
    
    
features = np.array(key_list[:1000])[:,0]
feature_matrix = []
cnt = 1
cores = multiprocessing.cpu_count()
pool = multiprocessing.Pool(processes=cores)
init_time = time.time()
for _ in pool.imap_unordered(pros, Cell_Phones_and_Accessories.iloc[:,3]):
    sys.stdout.write('done %d/%d time to go %d\r' % (cnt,len(Cell_Phones_and_Accessories), 
                                                     (len(Cell_Phones_and_Accessories)-cnt)/(time.time()-init_time)*cnt/60))
    cnt += 1
 

Process PoolWorker-44:
Process PoolWorker-43:
Process PoolWorker-41:
Process PoolWorker-42:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
    self.run()
    self.run()
    self.run()
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/process.py", line 114, in run
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/

  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/pool.py", line 113, in worker
    result = (True, func(*args, **kwds))
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/multiprocessing/pool.py", line 113, in worker
    result = (True, func(*args, **kwds))
    result = (True, func(*args, **kwds))
    result = (True, func(*args, **kwds))
  File "<ipython-input-23-0f22437a89ac>", line 17, in pros
  File "<ipython-input-23-0f22437a89ac>", line 17, in pros
  File "<ipython-input-23-0f22437a89ac>", line 17, in pros
    similar_words = model.most_similar(word)
  File "<ipython-input-23-0f22437a89ac>", line 17, in pros
    similar_words = model.most_similar(word)
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/site-packages/gensim/models/word2vec.py", line 1209, in most_similar
  File "/afs/inf.ed.ac.uk/user/s16/s1678999/miniconda2/envs/msc/lib/python2.7/site-packages/gensim/models/word2vec.

KeyboardInterrupt: 

In [17]:
import multiprocessing
cores = multiprocessing.cpu_count()
print cores

4


In [16]:

features = np.array(key_list[:1000])[:,0]
feature_matrix = []
with tqdm(total = len(Cell_Phones_and_Accessories)) as pbar:
    for review in Cell_Phones_and_Accessories.iloc[:,3]:
        review = review.lower()
        for punc in string.punctuation:
            review = review.replace(punc, ' ')
        review = nltk.word_tokenize(review)
        feature_vector = np.zeros(1000)
        feature_dict = {}
        for feature in features:
            feature_dict[feature] = 0
        for word in review:
            similar_words = []
            if word not in stopwords and len(word)>2:
                if model.wv.vocab.has_key(word):
                    similar_words = model.most_similar(word)
            for similar_word_pair in similar_words:
                similar_word = u''+similar_word_pair[0]
                if feature_dict.has_key(similar_word):
                    feature_dict[similar_word] += 1
        for i in range(len(features)):
            feature_vector[i] = feature_dict[features[i]]
        feature_matrix.append(feature_vector)
        pbar.update(1)

  1%|          | 2314/194439 [01:31<2:06:57, 25.22it/s]


KeyboardInterrupt: 

In [175]:
np.sum(feature_matrix)

29341446.0

In [176]:
np.save('word2vec_feature_matrix',feature_matrix)

In [178]:
np.savetxt('word2vec_feature_matrix.csv', feature_matrix, delimiter = ',')  